# Transformer

Transformer의 핵심 구조인 Multi-head Attention을 구현하는 실습입니다.
1. Multi-head attention 및 self-attention 구현.
2. 각 과정에서 일어나는 연산과 input/output 형태 이해.

### 필요 패키지 import

In [2]:
from torch import nn
from torch.nn import functional as F
from tqdm import tqdm

import torch
import math

## Req. 2-1 Multi-head self-attention 구조 익히기

### **데이터 전처리**
vocab_size 100인 가상의 시퀀스 데이터를 생성합니다. 

각 데이터에 할당된 숫자는 tokenizing과 정수화가 이뤄진 형태입니다.

In [3]:
pad_id = 0
vocab_size = 100

data = [
  [62, 13, 47, 39, 78, 33, 56, 13, 39, 29, 44, 86, 71, 36, 18, 75],
  [60, 96, 51, 32, 90],
  [35, 45, 48, 65, 91, 99, 92, 10, 3, 21, 54],
  [75, 51],
  [66, 88, 98, 47],
  [21, 39, 10, 64, 21],
  [98],
  [77, 65, 51, 77, 19, 15, 35, 19, 23, 97, 50, 46, 53, 42, 45, 91, 66, 3, 43, 10],
  [70, 64, 98, 25, 99, 53, 4, 13, 69, 62, 66, 76, 15, 75, 45, 34],
  [20, 64, 81, 35, 76, 85, 1, 62, 8, 45, 99, 77, 19, 43]
]

In [4]:
# 길이 맞춰주기 위해 패딩합니다.
def padding(data):
  max_len = len(max(data, key=len))
  print(f"Maximum sequence length: {max_len}")

  for i, seq in enumerate(tqdm(data)):
    if len(seq) < max_len:
      data[i] = seq + [pad_id] * (max_len - len(seq))

  return data, max_len

In [5]:
data, max_len = padding(data)

Maximum sequence length: 20


100%|██████████| 10/10 [00:00<00:00, 61052.46it/s]


In [6]:
data

[[62, 13, 47, 39, 78, 33, 56, 13, 39, 29, 44, 86, 71, 36, 18, 75, 0, 0, 0, 0],
 [60, 96, 51, 32, 90, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [35, 45, 48, 65, 91, 99, 92, 10, 3, 21, 54, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [75, 51, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [66, 88, 98, 47, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [21, 39, 10, 64, 21, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [98, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [77,
  65,
  51,
  77,
  19,
  15,
  35,
  19,
  23,
  97,
  50,
  46,
  53,
  42,
  45,
  91,
  66,
  3,
  43,
  10],
 [70, 64, 98, 25, 99, 53, 4, 13, 69, 62, 66, 76, 15, 75, 45, 34, 0, 0, 0, 0],
 [20, 64, 81, 35, 76, 85, 1, 62, 8, 45, 99, 77, 19, 43, 0, 0, 0, 0, 0, 0]]

### Hyperparameter 세팅 및 embedding

In [7]:
d_model = 512  # model의 hidden size
num_heads = 8  # head의 개수

# d_model이 입력을 projection 시킬 임베딩 space의 차원이므로, num_heads로 나누어 떨어져야 한다.

In [8]:
embedding = nn.Embedding(vocab_size, d_model)

# B: batch size, L: maximum sequence length
batch = torch.LongTensor(data)  # (B, L)
batch_emb = embedding(batch)  # (B, L, d_model)

In [9]:
print(batch_emb)
print(batch_emb.shape)

tensor([[[ 0.4131,  0.1323, -1.2643,  ..., -1.0075, -1.2104, -1.1472],
         [ 0.6792,  0.6466,  1.2484,  ..., -0.3469,  1.7738,  1.4269],
         [-0.8930,  1.7658,  0.0591,  ..., -0.0337, -0.6905,  1.3835],
         ...,
         [ 0.4488, -0.0648, -1.4498,  ...,  0.1912,  0.4578, -0.7364],
         [ 0.4488, -0.0648, -1.4498,  ...,  0.1912,  0.4578, -0.7364],
         [ 0.4488, -0.0648, -1.4498,  ...,  0.1912,  0.4578, -0.7364]],

        [[-1.3189, -0.2551,  0.0405,  ...,  0.0503, -0.0490,  0.1149],
         [ 0.2593, -0.8840,  0.1121,  ...,  1.3256,  1.0745,  0.1823],
         [ 0.3010, -0.0939, -0.2713,  ...,  1.3444, -0.0170,  0.5121],
         ...,
         [ 0.4488, -0.0648, -1.4498,  ...,  0.1912,  0.4578, -0.7364],
         [ 0.4488, -0.0648, -1.4498,  ...,  0.1912,  0.4578, -0.7364],
         [ 0.4488, -0.0648, -1.4498,  ...,  0.1912,  0.4578, -0.7364]],

        [[ 1.2501, -1.6768,  0.2525,  ...,  1.2778, -0.3592,  2.7070],
         [-0.5703,  0.3093,  0.1860,  ...,  0

### Linear projection & 여러 head로 나누기

Multi-head attention 내에서 쓰이는 linear projection matrix들을 정의합니다.

In [10]:
w_q = nn.Linear(d_model, d_model)
w_k = nn.Linear(d_model, d_model)
w_v = nn.Linear(d_model, d_model)

In [11]:
w_0 = nn.Linear(d_model, d_model)

In [12]:
q = w_q(batch_emb)  # (B, L, d_model)
k = w_k(batch_emb)  # (B, L, d_model)
v = w_v(batch_emb)  # (B, L, d_model)

print(q.shape)
print(k.shape)
print(v.shape)

torch.Size([10, 20, 512])
torch.Size([10, 20, 512])
torch.Size([10, 20, 512])


Q, k, v를 `num_head`개의 차원 분할된 여러 vector로 만듭니다.

- 이론적으로는 multi-head attention을 수행하면 input을 각각 다른 head 개수만큼의 Wq, Wk, Wv로 linear transformation 해서 각각 여러번의 attention 수행한 후 concat 한 후 linear transformation 수행해준다
- 구현에서는 Wq, Wk, Wv 한 개씩
- 실제 `attention is all you need` 논문의 구현 예시는 Query vector 한개를 dim으로 쪼개서 진행한다

In [13]:
batch_size = q.shape[0]
d_k = d_model // num_heads

# num_heads * d_k로 쪼갠다
q = q.view(batch_size, -1, num_heads, d_k)  # (B, L, num_heads, d_k)
k = k.view(batch_size, -1, num_heads, d_k)  # (B, L, num_heads, d_k)
v = v.view(batch_size, -1, num_heads, d_k)  # (B, L, num_heads, d_k)

print(q.shape)
print(k.shape)
print(v.shape)

torch.Size([10, 20, 8, 64])
torch.Size([10, 20, 8, 64])
torch.Size([10, 20, 8, 64])


In [14]:
# num_heads를 밖으로 뺌으로써
# 각 head가 (L, d_k) 만큼의 matrix를 가지고 self-attention 수행

q = q.transpose(1, 2)  # (B, num_heads, L, d_k)
k = k.transpose(1, 2)  # (B, num_heads, L, d_k)
v = v.transpose(1, 2)  # (B, num_heads, L, d_k)

print(q.shape)
print(k.shape)
print(v.shape)

torch.Size([10, 8, 20, 64])
torch.Size([10, 8, 20, 64])
torch.Size([10, 8, 20, 64])


### Scaled dot-product self-attention 구현

각 head에서 실행되는 self-attetion 과정입니다.

In [15]:
# shape - (L, L)
# 같은 sequence 내에 서로 다른 token들에게 얼마나 가중치를 두고 attention을 해야하는가
attn_scores = torch.matmul(q, k.transpose(-2, -1)) / math.sqrt(d_k)  # (B, num_heads, L, L)
# softmax - row-wise이기 때문에 dim은 -1
attn_dists = F.softmax(attn_scores, dim=-1)  # (B, num_heads, L, L)

print(attn_dists)
print(attn_dists.shape)

tensor([[[[0.0461, 0.0594, 0.0362,  ..., 0.0694, 0.0694, 0.0694],
          [0.0413, 0.0439, 0.0543,  ..., 0.0713, 0.0713, 0.0713],
          [0.0490, 0.0281, 0.0586,  ..., 0.0638, 0.0638, 0.0638],
          ...,
          [0.0404, 0.0582, 0.0322,  ..., 0.0601, 0.0601, 0.0601],
          [0.0404, 0.0582, 0.0322,  ..., 0.0601, 0.0601, 0.0601],
          [0.0404, 0.0582, 0.0322,  ..., 0.0601, 0.0601, 0.0601]],

         [[0.0496, 0.0408, 0.0351,  ..., 0.0652, 0.0652, 0.0652],
          [0.0746, 0.0298, 0.0781,  ..., 0.0422, 0.0422, 0.0422],
          [0.0416, 0.0720, 0.0406,  ..., 0.0364, 0.0364, 0.0364],
          ...,
          [0.0377, 0.0335, 0.0759,  ..., 0.0426, 0.0426, 0.0426],
          [0.0377, 0.0335, 0.0759,  ..., 0.0426, 0.0426, 0.0426],
          [0.0377, 0.0335, 0.0759,  ..., 0.0426, 0.0426, 0.0426]],

         [[0.0566, 0.0339, 0.0520,  ..., 0.0481, 0.0481, 0.0481],
          [0.0581, 0.0613, 0.0553,  ..., 0.0534, 0.0534, 0.0534],
          [0.0648, 0.0406, 0.0622,  ..., 0

In [16]:
attn_values = torch.matmul(attn_dists, v)  # (B, num_heads, L, d_k)

print(attn_values.shape)

torch.Size([10, 8, 20, 64])


### 각 head의 결과물 병합

각 head의 결과물을 concat하고 동일 차원으로 linear projection합니다.

In [17]:
attn_values = attn_values.transpose(1, 2)  # (B, L, num_heads, d_k)
attn_values = attn_values.contiguous().view(batch_size, -1, d_model)  # (B, L, d_model)

print(attn_values.shape)

torch.Size([10, 20, 512])


In [18]:
# w_0 : (d_model, d_model)
# 서로 다른 의미로 foucsing 된 각 head의 self-attention 정보들을 합쳐주는 역할 수행
outputs = w_0(attn_values)

print(outputs)
print(outputs.shape)

tensor([[[-0.0141, -0.1073,  0.0781,  ...,  0.0659,  0.1357,  0.0876],
         [-0.0086, -0.1131,  0.0404,  ...,  0.0483,  0.1232,  0.0951],
         [ 0.0463, -0.1708, -0.0037,  ...,  0.0350,  0.0904,  0.0354],
         ...,
         [ 0.0164, -0.1662,  0.0309,  ..., -0.0067,  0.1350,  0.0713],
         [ 0.0164, -0.1662,  0.0309,  ..., -0.0067,  0.1350,  0.0713],
         [ 0.0164, -0.1662,  0.0309,  ..., -0.0067,  0.1350,  0.0713]],

        [[-0.1126, -0.2621,  0.1964,  ...,  0.1528,  0.4709,  0.3580],
         [-0.0603, -0.2757,  0.1923,  ...,  0.1401,  0.4854,  0.3843],
         [-0.0072, -0.2726,  0.1661,  ...,  0.1287,  0.4862,  0.3150],
         ...,
         [-0.0451, -0.2731,  0.1589,  ...,  0.1569,  0.5206,  0.3342],
         [-0.0451, -0.2731,  0.1589,  ...,  0.1569,  0.5206,  0.3342],
         [-0.0451, -0.2731,  0.1589,  ...,  0.1569,  0.5206,  0.3342]],

        [[-0.0547, -0.0777,  0.0658,  ...,  0.0826,  0.3953,  0.2767],
         [ 0.0202, -0.0791,  0.0453,  ...,  0

## Req. 2-2 Multi-head self-attention 모듈 클래스 구현

위의 과정을 모두 합쳐 하나의 Multi-head attention 모듈 class를 구현하겠습니다.

아래 코드의 TODO 부분을 채워주세요.

In [19]:
class MultiheadAttention(nn.Module):
  def __init__(self):
    super(MultiheadAttention, self).__init__()

    # Q, K, V learnable matrices
    self.w_q = nn.Linear(d_model, d_model)
    self.w_k = nn.Linear(d_model, d_model)
    self.w_v = nn.Linear(d_model, d_model)

    # Linear projection for concatenated outputs
    self.w_0 = nn.Linear(d_model, d_model)

  # scaled-dot product attention
  def self_attention(self, q, k, v):
    attn_scores = torch.matmul(q, k.transpose(-2, -1)) / math.sqrt(d_k)  # (B, num_heads, L, L)
    attn_dists = F.softmax(attn_scores, dim=-1)  # (B, num_heads, L, L)

    attn_values = torch.matmul(attn_dists, v)  # (B, num_heads, L, d_k)

    return attn_values

  def forward(self, q, k, v):
    batch_size = q.shape[0]

    # linear projection
    ################################################################################
    # TODO 1: Implement the forward pass for linear projection.                #
    ################################################################################
    # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
    q = w_q(batch_emb)
    k = w_k(batch_emb)
    v = w_v(batch_emb)


    # head만큼 쪼개준다
    ################################################################################
    # TODO 2: Implement the forward pass for split head.                #
    ################################################################################
    # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
    q = q.view(batch_size, -1, num_heads, d_k)  
    k = k.view(batch_size, -1, num_heads, d_k)  
    v = v.view(batch_size, -1, num_heads, d_k) 
    

    # 각 head가 (L, d_k)의 matrix를 담당하도록 만든다
    q = q.transpose(1, 2)  # (B, num_heads, L, d_k)
    k = k.transpose(1, 2)  # (B, num_heads, L, d_k)
    v = v.transpose(1, 2)  # (B, num_heads, L, d_k)

    attn_values = self.self_attention(q, k, v)  # (B, num_heads, L, d_k)
    attn_values = attn_values.transpose(1, 2).contiguous().view(batch_size, -1, d_model)  # (B, L, num_heads, d_k) => (B, L, d_model)

    return self.w_0(attn_values)

In [20]:
multihead_attn = MultiheadAttention()

outputs = multihead_attn(batch_emb, batch_emb, batch_emb)  # (B, L, d_model)

In [21]:
print(outputs)
print(outputs.shape)  # (batch_size, length, d_model)

tensor([[[-1.5063e-01,  5.7255e-02,  1.2729e-01,  ..., -7.3855e-02,
           7.0249e-02,  1.3474e-01],
         [-1.5924e-01,  8.6500e-02,  1.7459e-01,  ..., -8.9794e-02,
           8.0234e-02,  1.4159e-01],
         [-1.5543e-01,  1.0135e-01,  1.5755e-01,  ..., -1.7906e-02,
           8.8580e-02,  1.4999e-01],
         ...,
         [-2.4042e-01,  6.5054e-02,  1.9314e-01,  ..., -2.5424e-02,
           8.0621e-02,  1.7718e-01],
         [-2.4042e-01,  6.5054e-02,  1.9314e-01,  ..., -2.5424e-02,
           8.0621e-02,  1.7718e-01],
         [-2.4042e-01,  6.5054e-02,  1.9314e-01,  ..., -2.5424e-02,
           8.0621e-02,  1.7718e-01]],

        [[-1.2440e-01, -1.2262e-01,  3.3839e-02,  ..., -1.2694e-01,
           3.7475e-01,  2.4266e-02],
         [-1.6041e-01, -9.5337e-02,  2.6799e-02,  ..., -1.7088e-01,
           4.1076e-01, -2.6719e-02],
         [-1.5907e-01, -9.1937e-02,  1.9654e-02,  ..., -1.9577e-01,
           3.8025e-01, -1.7951e-02],
         ...,
         [-1.5510e-01, -1